In [1]:
# route = "M15"

In [2]:
# import Pkg
# Pkg.add("CSV")
# Pkg.add("DataFrames")
# Pkg.add("DataConvenience")
# Pkg.add("Parquet")
# Pkg.add("HDF5")
# Pkg.add("JuliaDB")

In [3]:
# using CSV
# using DataFrames

# df = CSV.read("data/nyc_buses_with_passenger_counts_20210401_20210630.csv", missingstring="\\N\", DataFrame)

# # filter out rows without our route
# df = filter(row -> row.route_short != route, df)

# # write parquet
# using Parquet
# file = "data/nyc_buses_with_passenger_counts_20210401_20210630_$route.julia.parquet"
# write_parquet(file, df)

## STREAMING VERSION

In [4]:
using DataFrames
using CSV
using DataConvenience
using Parquet

filepath = "data/nyc_buses_with_passenger_counts_20210401_20210630.csv"

master_df = DataFrame()

i=0

for chunk in CsvChunkIterator(filepath, missingstring="\\N", limit=1000000)
    
    # 
#     # filter out rows without our route
#     chunk = filter(row -> row.route_short == route, chunk)
    n = nrow(chunk)
    println("Extracted $n rows in chunk $i")
    
    # concatenate to the master df
    master_df  = vcat(master_df, chunk)
    
    i += 1
    
end

Extracted 1000000 rows in chunk 0
Extracted 1000000 rows in chunk 1
Extracted 1000000 rows in chunk 2
Extracted 1000000 rows in chunk 3
Extracted 1000000 rows in chunk 4
Extracted 1000000 rows in chunk 5
Extracted 1000000 rows in chunk 6
Extracted 1000000 rows in chunk 7
Extracted 1000000 rows in chunk 8
Extracted 1000000 rows in chunk 9
Extracted 1000000 rows in chunk 10
Extracted 1000000 rows in chunk 11
Extracted 1000000 rows in chunk 12
Extracted 1000000 rows in chunk 13
Extracted 1000000 rows in chunk 14
Extracted 1000000 rows in chunk 15
Extracted 1000000 rows in chunk 16
Extracted 1000000 rows in chunk 17
wrote to data/citywide_test_julia.parquet


LoadError: "Column whose `eltype` is String31 is not supported at this stage. \nColumn whose `eltype` is Dates.Date is not supported at this stage. \nColumn whose `eltype` is String7 is not supported at this stage. \nColumn whose `eltype` is String15 is not supported at this stage. \nColumn whose `eltype` is Union{Missing, String15} is not supported at this stage. \n"

In [11]:
describe(master_df)

,variable,mean,min,median,max
,Symbol,Union…,Any,Any,Any
1,timestamp,,2021-03-31 23:58:17,,2021-07-01 04:58:17
2,service_date,,2021-04-01,2021-05-17,2021-06-30
3,hour,11.7952,0,11.0,23
4,minute,29.3036,0,29.0,59
5,route_short,,B1,,X68
6,direction,0.500526,0,1.0,1
7,vehicle_id,,MTA NYCT_1000,,MTA NYCT_9721
8,lat,40.7278,40.5029,40.7169,40.9124
9,lon,-73.9284,-74.2523,-73.9344,-73.7012


# TODO fix data types

In [15]:
mapcols(eltype, master_df)

,timestamp,service_date,hour,minute,route_short,direction,vehicle_id
,DataType,DataType,DataType,DataType,DataType,DataType,DataType
1,String31,Date,Int64,Int64,String7,Int64,String15


In [ ]:
# write to a new file
file = string("data/citywide_test_julia.parquet")
print("wrote to $file\n")
write_parquet(file, master_df)